In [4]:
# Clone your GitHub repo
#!git clone https://github.com/joachimasare/persistent-generative-scenes.git
%cd persistent-generative-scenes

# Install dependencies
!pip install -r requirements.txt


[WinError 2] The system cannot find the file specified: 'persistent-generative-scenes'
C:\Users\asare\Documents\GitHub\food-funding-app\persistent-generative-scenes
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Cell 1: 📁 Jump to project root & expose it to Python

import os, sys

# If we're inside the notebooks/ folder, go up one level:
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(os.path.join(os.getcwd(), os.pardir))

# Now cwd should be the folder that contains src/, scripts/, requirements.txt
print("Project root is:", os.getcwd())

# Prepend it to sys.path so `import src...` and `import scripts...` work:
sys.path.insert(0, os.getcwd())
print("sys.path[0] =", sys.path[0])


Project root is: C:\Users\asare\Documents\GitHub\food-funding-app\persistent-generative-scenes
sys.path[0] = C:\Users\asare\Documents\GitHub\food-funding-app\persistent-generative-scenes


In [4]:

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from src.memory.voxel_memory import VoxelMemory
from scripts.gen_scenes import create_block_world, render_scene

# Camera intrinsic parameters
width, height = 256, 256
fx = fy = width
cx, cy = width / 2, height / 2
class Intrinsic:
    def __init__(self, fx, fy, cx, cy):
        self.fx, self.fy, self.cx, self.cy = fx, fy, cx, cy
intrinsic = Intrinsic(fx, fy, cx, cy)

ModuleNotFoundError: No module named 'src.memory'

In [ ]:
# Cell 3: Generate scene and render RGB-D
mesh = create_block_world(size=16, num_blocks=30)
pose = np.eye(4)
pose[:3,3] = [30, 30, 30]
rgb, depth = render_scene(mesh, pose, width=width, height=height)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title("RGB")
plt.imshow(rgb)
plt.axis('off')
plt.subplot(1,2,2)
plt.title("Depth")
plt.imshow(depth, cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
# Cell 4: Update VoxelMemory and query depth hint
mem = VoxelMemory(dims=(32,32,32), voxel_size=1.0)
mem.update(pose, rgb, depth, intrinsic)
depth_hint = mem.query(pose, intrinsic, view_size=(height, width))

plt.figure(figsize=(4,4))
plt.title("Depth Hint")
plt.imshow(depth_hint, cmap="viridis")
plt.axis('off')
plt.show()
```

```python
# Cell 5: Save ground-truth and hint maps
import os
os.makedirs("data/phase1", exist_ok=True)
np.save("data/phase1/depth_gt.npy", depth)
np.save("data/phase1/depth_hint.npy", depth_hint)